# transformers
-  Hugging Face에서 제공하는 NLP 모델 라이브러리로, BERT, GPT, RoBERTa, T5 등 다양한 사전 학습된 모델을 지원한다.
- NLP와 트랜스포머 기반 딥러닝 연구를 간편하게 수행할 수 있도록 도와준다.

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,144 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123634 files and dire

In [1]:
import matplotlib.pyplot as plt
plt.rc('font',family="NanumBarunGothic")
plt.rcParams["axes.unicode_minus"] = False



```
gdown : google drive에 저장된 파일 다운로드할 수 있게 도와주는 패키지  
(머신러닝 모델, 데이터셋)  
einops : 딥러닝 작업에서 텐서의 재배열을 간단하게 처리하도록 도와주는 패키지  
(transformers 모델처럼 복잡한 텐서 구조에 유용)  
sentencepiece : 텍스트 토큰화, 언어에 구애받지 않고 BPE(Byte Pair Encoding)나 Unigram 모델 기반으로 서브워드 토큰화 제공  
(Hugging Face와 같은 NLP 라이브러리와 함께 사용)  
sacremoses : 텍스트 전처리, 텍스트를 토큰화하거나 역토큰화할 때 사용되며, 번역 모델의 전처리 단계에서 자주 사용
```



In [2]:
!pip install gdown
!pip inatall einops
!pip install sentencepiece
!pip install transformers
!pip install sacremoses

ERROR: unknown command "inatall" - maybe you meant "install"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.4 MB/s eta 0:00:00


In [3]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#
train_model = False

DEVICE

'cuda'

# 1. 모델 구현  
## 1. MHA ( Multi Head Attention )  
Encoder, Decoder의 Self Attention을 수행하는 모듈  
### 1.1 einops 사용하기  
- 장점 : 텐서의 변환을 매우 쉽게 이해하고 변환 가능  
- 단점 : 오류 잡기가 매우 힘들고 속도도 비교적 느리다

In [4]:
from einops import rearrange

# 차원 재배치
x = torch.randn(2, 3, 4) # Tensor shape : (2, 3, 4)
y = rearrange(x, 'a b c -> c a b')

y.shape


torch.Size([4, 2, 3])

In [6]:
# 차원 합치기
x = torch.randn(2, 3, 4)
y = rearrange(x, 'a b c -> (a b) c')

y.shape

torch.Size([6, 4])

In [7]:
# 차원 나누기
x = torch.randn(6, 4)
y = rearrange(x, '(a b) c -> a b c', a=2, b=3)

y.shape

torch.Size([2, 3, 4])

## 2. MHA 구현

In [8]:
from torch import nn
from einops import rearrange
import torch

class MHA(nn.Module):

  def __init__(self, d_model=512, n_heads=8):
    """
      d_model : 임베딩 벡터의 차원 (dimension of embedding vector)
      n_heads : 멀티 헤드 어텐션의 헤드 개수 (number of heads in multi_head attention)
    """
    super().__init__()

    self.n_heads = n_heads

    # Query, Key, Value 벡터를 위한 선형 레이어를 정의
    self.fc_q = nn.Linear(d_model, d_model)
    self.fc_k = nn.Linear(d_model, d_model)
    self.fc_v = nn.Linear(d_model, d_model)

    # 최종 출력을 위한 선형 레이어
    self.fc_o = nn.Linear(d_model, d_model)

    # 어텐션 스코어를 스케일링 하기 위한 값. (d_model / n_heads의 제곱근)
    self.scale = torch.sqrt(torch.tensor(d_model / n_heads)) # 512 / 8의 root

SyntaxError: incomplete input (<ipython-input-8-bf6a5b84437a>, line 8)